In [1]:
import os
os.environ['PYKX_4_1_ENABLED'] = "True"
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


## --- Day 9: Movie Theater --- [link](https://adventofcode.com/2025/day/9)

You slide down the firepole in the corner of the playground and land in the North Pole base movie theater!

The movie theater has a big tile floor with an interesting pattern. Elves here are redecorating the theater by switching out some of the square tiles in the big grid they form. Some of the tiles are red; the Elves would like to find the largest rectangle that uses red tiles for two of its opposite corners. They even have a list of where the red tiles are located in the grid (your puzzle input).

In [86]:
show reds:get each read0`:test.txt

7  1
11 1
11 7
9  7
9  5
2  5
2  3
7  3


In [87]:
/part 1
area:{[(p1;p2)] prd 1+abs .[-]reds(p1;p2)}
/ max area over all non-repeating pairs
max area@'pairs:{l where (<).'l:x cross x} til count reds

50


In [88]:
/ part 2
/ tricky polygon shape, good thing - orthogonal
/ so each candidate rectangle needs to be checked for
/ 1) insides of rectangle being inside polygon - use raytracing
/ 2) polygon edges not intersecting rectangle edges

show edges:{2#x rotate reds} each til count reds / edges of polygon
show "-"
show vedges:edges where (=).'edges[;0 1;0] / vertical edges
show "-"
show hedges:edges where (<>).'edges[;0 1;0] / vertical edges

7  1 11 1
11 1 11 7
11 7 9  7
9 7  9 5 
9 5  2 5 
2 5  2 3 
2 3  7 3 
7 3  7 1 
"-"
11 1 11 7
9 7  9 5 
2 5  2 3 
7 3  7 1 
"-"
7  1 11 1
11 7 9  7
9 5  2 5 
2 3  7 3 


In [89]:
/ raytrace - even number of edge crossings - outside, odd - inside
show xmax:1+max reds[;0]

ray:{[(start;stop)] start+til 1+`long$stop-start}

sum any 1_'differ each ray[(1.5;xmax+0.5)]</:1 2 3

2i
12


In [90]:
/ edge intersection check
edgeInter:{[(r1;r2);(p1;p2)] 
    / 0b - no intersection
    (minpx;minpy):min(p1;p2);
    (maxpx;maxpy):max(p1;p2);
    (minrx;minry):min(r1;r2);
    (maxrx;maxry):max(r1;r2);
    if[minpx=maxpx; / vertical edge
        :$[(minpx<=minrx)|minpx>=maxrx; / not within rectangle x-boundaries
            0b;
            max[(minpy;minry)]<min(maxpy;maxry) / otherwise - check y overlap
            ];
     ];
    if[minpy=maxpy; / horizontal edge
        :$[(minpy<=minry)|minpy>=maxry; / not within rectangle y-boundaries
            0b;
            max[(minpx;minrx)]<min(maxpx;maxrx) / otherwise - check x overlap
            ];
     ];
 };
edgeInter[(2 3;11 7);0N!vedges 1]

1b
(9 7;9 5)


In [91]:
edgeInter[(2 3;9 5)] each edges

00000000b


In [92]:
show insidePairs:pairs where {[(i1;i2)] / indicies in red tiles list
    / rect diag coords, top left corner and interior point
    (ipx;ipy):0.5+min(r1;r2):reds(i1;i2);
    / horizontal ray from internal point
    ihr:ray(ipx;0.5+xmax);
    / vertical edges on the same y-level as internal point
    vex:vedges[;0;0] where ipy within/:vedges[;0 1;1];
    / if interior point not inside - exit early
    if[0=(sum any 1_'differ each ihr</:vex) mod 2;:0b];
    / check all edges strictly outside rectangle boundaries
    not any edgeInter[(r1;r2)] each edges
 } each pairs

0 1
0 4
0 6
0 7
1 3
1 4
1 7
2 4
3 4
3 5
4 5
4 6
4 7
5 6
5 7
6 7


In [93]:
max area@'insidePairs

24
